In [8]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
from datetime import datetime, timedelta

In [465]:
class visitClean:
    def __init__(self, data):
        self.data = data

    def getDate(self, year, week):
        if week == 1:
            r = year % 7
            d = (12 - r) % 7
            if d == 0:
                return datetime(year, 6, 7)
            else:
                return datetime(year, 6, d)
        else:
            return self.getDate(year, week - 1) + timedelta(weeks = 1)

    def fromWeeksToDates(self):
        self.data['Dates'] = self.data.apply(lambda row: self.getDate(row['Year'], row['Week']), axis=1)
        self.data = self.data.drop(columns=['Year', 'Week'])
        self.data = self.data[['Dates'] + [col for col in self.data.columns if col != 'Dates']]

    def getResort(self, i):
        self.data = self.data.iloc[:, [0, i]]

In [505]:
class climateClean:
    def __init__(self, data):
        self.data = data

    def fromYMDtoDates(self):
        self.data['Dates'] = self.data.apply(lambda row: datetime(int(row['Year']), int(row['Month']), int(row['Day'])), axis=1)
        self.data.drop(columns=['Year', 'Month', 'Day'], inplace=True)
        self.data = self.data[['Dates'] + [col for col in self.data.columns if col != 'Dates']]

    def filterStation(self, station):
        self.data = self.data[self.data['Bureau of Meteorology station number'] == station]

    def keepRelevantData(self, data):
        self.data = self.data[self.data['Dates'].isin(data['Dates'])]

    def combineData(self, data):
        self.data = pd.DataFrame(np.hstack([data, self.data.drop(['Dates'], axis = 1)]))

    

In [507]:
xls = pd.ExcelFile("2025 Allianz Datathon Dataset.xlsx")
xls.sheet_names

['readme', 'Visitation Data', 'Climate Data']

In [508]:
visitDataRaw = xls.parse('Visitation Data')
baw = visitClean(visitDataRaw)

In [509]:
baw.fromWeeksToDates()

In [510]:
baw.getResort(1)


In [511]:
baw.data

,Dates,Mt. Baw Baw
0,2014-06-07,555
1,2014-06-14,804
2,2014-06-21,993
3,2014-06-28,2976
4,2014-07-05,11112
...,...,...
160,2024-08-13,5977
161,2024-08-20,3597
162,2024-08-27,1500
163,2024-09-03,0


In [517]:
climateDataRaw = xls.parse('Climate Data')
climateData = climateClean(climateDataRaw)
climateData.filterStation(85291)

In [518]:
climateData.fromYMDtoDates()

/var/folders/st/vt6hxj_93517y4vp9mxflvn80000gn/T/ipykernel_99215/84740483.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Dates'] = self.data.apply(lambda row: datetime(int(row['Year']), int(row['Month']), int(row['Day'])), axis=1)
/var/folders/st/vt6hxj_93517y4vp9mxflvn80000gn/T/ipykernel_99215/84740483.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.drop(columns=['Year', 'Month', 'Day'], inplace=True)


In [519]:
climateData.keepRelevantData(baw.data)

In [520]:
climateData.data

,Dates,Bureau of Meteorology station number,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
30055,2014-06-07,85291,3.6,-0.5,3.6
30062,2014-06-14,85291,3.9,1.9,3.0
30069,2014-06-21,85291,4.8,1.6,0.0
30076,2014-06-28,85291,-0.6,-1.4,NaN
30083,2014-07-05,85291,0.9,-0.6,3.4
...,...,...,...,...,...
33775,2024-08-13,85291,8.1,3.1,0.0
33782,2024-08-20,85291,4.9,2.6,0.0
33789,2024-08-27,85291,6.9,1.6,0.0
33796,2024-09-03,85291,5.8,-3.9,6.0


In [521]:
climateData.combineData(baw.data)

In [529]:
pd.DataFrame(climateData.data).drop(2, axis = 1)

,0,1,3,4,5
0,2014-06-07,555,3.6,-0.5,3.6
1,2014-06-14,804,3.9,1.9,3.0
2,2014-06-21,993,4.8,1.6,0.0
3,2014-06-28,2976,-0.6,-1.4,NaN
4,2014-07-05,11112,0.9,-0.6,3.4
...,...,...,...,...,...
160,2024-08-13,5977,8.1,3.1,0.0
161,2024-08-20,3597,4.9,2.6,0.0
162,2024-08-27,1500,6.9,1.6,0.0
163,2024-09-03,0,5.8,-3.9,6.0
